In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
!git clone https://github.com/chiruu12/NexNet.git
    

Cloning into 'NexNet'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 173 (delta 84), reused 122 (delta 50), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 889.78 KiB | 6.84 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [3]:
import os
os.chdir('NexNet')

In [4]:
from Models import FNN,CNN
from Losses import CrossEntropyLoss
from Layers import Linear
from Activation_classes import ReLu,Softmax,PReLU
from utils import one_hot,Initializer
from Optimizer import Adam,AdaDelta

In [5]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
submission=pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y_train=train["label"].to_numpy()
X_train=train.drop(columns=['label']).to_numpy()


In [7]:
# Prepare the data (assuming X_train, y_train, X_val, y_val are already defined)
from sklearn.model_selection import train_test_split

# Example data split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

# One-hot encode the labels
num_classes = 10  # For MNIST
one_hot_encoder=one_hot(num_classes)
y_train_one_hot = one_hot_encoder.convert_to_one_hot(y_train)
y_test_one_hot = one_hot_encoder.convert_to_one_hot(y_test)
# Create the model
input_dim = X_train.shape[1]
model = FNN(optimizer=AdaDelta(),loss=CrossEntropyLoss())

# Add layers to the model
model.add_layer(Linear(input_dim=input_dim,output_dim=128,activation=PReLU()))
model.add_layer(Linear(input_dim=128,output_dim=128,activation=PReLU()))
model.add_layer(Linear(input_dim=128,output_dim=32,activation=PReLU()))
model.add_layer(Linear(32, num_classes))

# Train the model
model.train(X_train, y_train_one_hot, epochs=30, batch_size=64)

# Evaluate on test set
accuracy = model.evaluate(X_test,y_test_one_hot)



Epoch 1/30, ---- Total Loss: 613.5300, ---- Average Loss: 1.1686
Epoch 2/30, ---- Total Loss: 178.6647, ---- Average Loss: 0.3403
Epoch 3/30, ---- Total Loss: 113.3189, ---- Average Loss: 0.2158
Epoch 4/30, ---- Total Loss: 85.0862, ---- Average Loss: 0.1621
Epoch 5/30, ---- Total Loss: 67.2087, ---- Average Loss: 0.1280
Epoch 6/30, ---- Total Loss: 54.3965, ---- Average Loss: 0.1036
Epoch 7/30, ---- Total Loss: 47.7172, ---- Average Loss: 0.0909
Epoch 8/30, ---- Total Loss: 39.7381, ---- Average Loss: 0.0757
Epoch 9/30, ---- Total Loss: 33.9826, ---- Average Loss: 0.0647
Epoch 10/30, ---- Total Loss: 31.2120, ---- Average Loss: 0.0595
Epoch 11/30, ---- Total Loss: 25.7911, ---- Average Loss: 0.0491
Epoch 12/30, ---- Total Loss: 23.9223, ---- Average Loss: 0.0456
Epoch 13/30, ---- Total Loss: 22.1788, ---- Average Loss: 0.0422
Epoch 14/30, ---- Total Loss: 18.4852, ---- Average Loss: 0.0352
Epoch 15/30, ---- Total Loss: 17.7331, ---- Average Loss: 0.0338
Epoch 16/30, ---- Total Loss: 1

96.88095238095238

In [8]:
model.train(X_test, y_test_one_hot, epochs=20, batch_size=64)
model.evaluate(X_test, y_test_one_hot)

Epoch 1/20, ---- Total Loss: 22.6868, ---- Average Loss: 0.1719
Epoch 2/20, ---- Total Loss: 11.3104, ---- Average Loss: 0.0857
Epoch 3/20, ---- Total Loss: 6.7552, ---- Average Loss: 0.0512
Epoch 4/20, ---- Total Loss: 4.5905, ---- Average Loss: 0.0348
Epoch 5/20, ---- Total Loss: 2.6828, ---- Average Loss: 0.0203
Epoch 6/20, ---- Total Loss: 2.6617, ---- Average Loss: 0.0202
Epoch 7/20, ---- Total Loss: 1.7273, ---- Average Loss: 0.0131
Epoch 8/20, ---- Total Loss: 1.2487, ---- Average Loss: 0.0095
Epoch 9/20, ---- Total Loss: 0.9658, ---- Average Loss: 0.0073
Epoch 10/20, ---- Total Loss: 0.9683, ---- Average Loss: 0.0073
Epoch 11/20, ---- Total Loss: 0.6153, ---- Average Loss: 0.0047
Epoch 12/20, ---- Total Loss: 0.8529, ---- Average Loss: 0.0065
Epoch 13/20, ---- Total Loss: 0.6122, ---- Average Loss: 0.0046
Epoch 14/20, ---- Total Loss: 0.2455, ---- Average Loss: 0.0019
Epoch 15/20, ---- Total Loss: 2.3685, ---- Average Loss: 0.0179
Epoch 16/20, ---- Total Loss: 0.5644, ---- Aver

In [9]:
test_array=model.predict(test)
y_ans=one_hot.one_hot_to_label(test_array)
ans=pd.DataFrame({
    'ImageId':submission["ImageId"].to_numpy(),
    'Label':y_ans
})

In [10]:
ans.to_csv('/kaggle/working/submission.csv',index=False)
